# Finding the nearest train/tube station

The purpose of this project is to create module that takes a London postcode input e.g. N4 4AF and will output the nearest train station to it.

In [1]:
import pandas as pd
import requests
from geopy import distance

In [2]:
# A csv file of london stations can be downloaded from this link
# https://www.doogal.co.uk/london_stations.php
# TODO change to all gb stations so it is not london centric

london_stations = pd.read_csv('london_stations.csv')

In [3]:
london_stations.head()

,Station,OS X,OS Y,Latitude,Longitude,Zone,Postcode
0,Abbey Road,539081,183352,51.531952,0.003723,3,E15 3NB
1,Abbey Wood,547297,179002,51.490784,0.120272,4,SE2 9RH
2,Acton Central,520613,180299,51.508758,-0.263430,2,W3 6BH
3,Acton Main Line,520296,181196,51.516887,-0.267690,3,W3 9EH
4,Acton Town,519457,179639,51.503071,-0.280303,3,W3 8HN


In [4]:
test_postcodes = ['N15 3AD', 'N4 4AF', 'N10 3QS', 'N15 4AR', 'N15 4JF']

In [5]:
# 1. Geocode postcode data using api

# create endpoint
def create_endpoint(postcode):
    postcode = postcode.replace(' ','').lower()
    output = f'http://api.postcodes.io/postcodes/{postcode}'
    return output

In [6]:
create_endpoint(test_postcodes[0])

'http://api.postcodes.io/postcodes/n153ad'

In [25]:
# Get longitude and latitude for the code
def get_long_lat(postcode):
    endpoint = create_endpoint(postcode)
    output_dict = {}
    r = requests.get(endpoint)
    output_dict['postcode'] = r.json()['result']['postcode']
    output_dict['lat_lon'] = (r.json()['result']['latitude'], r.json()['result']['longitude'])
    return output_dict

In [26]:
test_dict = get_long_lat(test_postcodes[0])

In [27]:
test_dict

{'postcode': 'N15 3AD', 'lat_lon': (51.583631, -0.092808)}

In [33]:
london_stations[['Station', 'Latitude', 'Longitude']][london_stations['Station'].isin(['Blackhorse Road', 'Walthamstow Central', 'Tottenham Hale'])]

,Station,Latitude,Longitude
60,Blackhorse Road,51.585777,-0.039626
562,Tottenham Hale,51.587998,-0.060188
586,Walthamstow Central,51.583018,-0.019941


In [19]:
# measure distance between two points

# test with postcode
# distance.distance((lat, lon), (lat, lon))

print(distance.distance(test_dict['lat_lon'], (51.587998, -0.060188)).miles)
print(distance.distance(test_dict['lat_lon'], (51.587998, -0.060188)).km)

1.4369277331214914
2.3125110257326735


In [34]:
london_stations[['Station', 'Latitude', 'Longitude']][london_stations['Station'].isin(['Blackhorse Road', 'Walthamstow Central', 'Tottenham Hale'])]

,Station,Latitude,Longitude
60,Blackhorse Road,51.585777,-0.039626
562,Tottenham Hale,51.587998,-0.060188
586,Walthamstow Central,51.583018,-0.019941


In [32]:
london_stations[london_stations['Station'].isin(['Blackhorse Road', 'Walthamstow Central', 'Tottenham Hale'])].values[0]

array(['Blackhorse Road', 535914, 189256, 51.5857766270677,
       -0.0396257115532134, '3', 'E17 6JJ'], dtype=object)

In [30]:
# TODO Create a function to iterate through a list of stations and return that data

In [ ]:
# TODO Get the smallest distance from the list of distances